In [14]:
import os
import pandas as pd
from itables import init_notebook_mode
import yaml
import time
from datetime import datetime


from IPython.display import display, clear_output



STATUS_DIR = "/home/hpinkard_waller/GitRepos/microscoBayes/experiments/config_files/"
SAVING_DIR_ROOT = "/home/hpinkard_waller/models/"


names = []

complete = os.listdir(STATUS_DIR + 'complete')
training = os.listdir(STATUS_DIR + 'training')
pending = os.listdir(STATUS_DIR + 'pending')

config_files = complete + training + pending
statuses = len(complete) * ["complete"] + len(training) * ["training"] + len(pending) * ["pending"]


# Read stuff from its config file
tensorboard_dirs = []
config_paths = []
dates = []
elapsed_times = []
attempts = []
for config_file, status in zip(config_files, statuses):
    config_file_path = STATUS_DIR + status + '/' + config_file
    with open(config_file_path, "r") as stream:
        config = yaml.safe_load(stream)
        m_time = os.path.getmtime(config_file_path)
        
    if 'scheduler' not in config or 'date' not in config['scheduler']:
        dates.append(datetime.fromtimestamp(m_time).strftime("%Y-%m-%d"))
    else:
        dates.append(config['scheduler']['date'])

    if 'training' in config:
        if 'tensorboard_dir' not in config['training']:
            tensorboard_dirs.append('NA')
        else:
            tensorboard_dirs.append(config['training']['tensorboard_dir'])
    else:
        tensorboard_dirs.append('pending')
    if status == 'complete':
        if 'elapsed' not in config['training']:
            elapsed_times.append(None)
        else:
            elapsed_times.append(config['training']['elapsed'])
    elif 'training' not in config:
        elapsed_times.append(None)
    else:
        elapsed_times.append(time.time() - config['training']['start_time'])
    if 'scheduler' not in config or 'attempt_number' not in config['scheduler']:
        attempts.append('NA')
    else:
        attempts.append(int(config['scheduler']['attempt_number']))

    if (elapsed_times[-1] is not None):
        # format date string
        days = f"{int(elapsed_times[-1] // (24 * 60**2))} days  "
        hours, remainder = divmod(elapsed_times[-1], 3600)
        minutes, seconds = divmod(remainder, 60)
        formatted_time = '{}:{:02d}:{:02d}'.format(int(hours), int(minutes), int(seconds))
        elapsed_times[-1] = formatted_time

    config_paths.append(STATUS_DIR.replace(os.path.expanduser('~'), '') + status + '/' + config_file)


d = {"date": dates, "name": [cf.replace('_', " ") for cf in config_files],
     "status": statuses, "attempts": attempts,  "elapsed_time": elapsed_times,
    #  "config": config_paths, 
     "tensorboard": tensorboard_dirs}
df = pd.DataFrame(data=d)
# df.style.set_properties(subset=['config_files'], **{'min-width': '500px'})
pd.options.display.min_rows = 100



df['status'] = pd.Categorical(df['status'], ["complete", "training", "pending"])
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['status', 'date'])

def stylize(x):
    if x == 'complete':
        return 'color: green'
    elif x == 'training':
        return 'color: orange'
    elif x == 'pending':
        return 'color: blue'
    return None
    

df = df.style.applymap(stylize)  

    
display(df)


,date,name,status,attempts,elapsed_time,tensorboard
6,2023-01-09,logistic.yaml,complete,NA,3:28:23,/home/hpinkard_waller/2tb_ssd/models/logistic/tensorboard/
9,2023-01-09,gaussian.yaml,complete,NA,4:07:52,/home/hpinkard_waller/2tb_ssd/models/gaussian/tensorboard/
13,2023-01-10,marginals mixture density all markers.yaml,complete,NA,4:09:03,/home/hpinkard_waller/2tb_ssd/models/marginals_mixture_density_all_markers/tensorboard/
4,2023-01-26,double descent test big model.yaml,complete,NA,68:18:02,/home/hpinkard_waller/2tb_ssd/models/double_descent_test_big_model/tensorboard/
0,2023-01-25,marginals scalar scalar hparams CD3 finetune.yaml,complete,NA,125:13:20,/home/hpinkard_waller/2tb_ssd/models/marginals_scalar_scalar_hparams_CD3_finetune/tensorboard/
1,2023-01-25,marginals mixture density CD3 only.yaml,complete,NA,52:28:34,/home/hpinkard_waller/2tb_ssd/models/marginals_mixture_density_CD3_only/tensorboard/
2,2023-01-25,marginals mixture density CD3 only LR3e-5.yaml,complete,NA,7:01:30,/home/hpinkard_waller/2tb_ssd/models/marginals_mixture_density_CD3_only_LR3e-5/tensorboard/
3,2023-01-25,marginals scalar scalar hparams CD3 only.yaml,complete,NA,24:04:35,/home/hpinkard_waller/2tb_ssd/models/marginals_scalar_scalar_hparams_CD3_only/tensorboard/
5,2023-01-25,marginals scalar density hparams CD3 finetune.yaml,complete,NA,37:24:41,/home/hpinkard_waller/2tb_ssd/models/marginals_scalar_density_hparams_CD3_finetune/tensorboard/
7,2023-01-25,marginals scalar scalar hparams CD3 early stopping.yaml,complete,NA,3:40:21,/home/hpinkard_waller/2tb_ssd/models/marginals_scalar_scalar_hparams_CD3_early_stopping/tensorboard/
